In [8]:
import networkx as nx
import ba_utils.visualization as visualization
import ba_utils.orderings as orderings
import ba_utils.network_rugs as nrugs
import os
import json

In [ ]:
def load_real_network(filename="data/contact_pattern_2012.json"):
    """
    Load a temporal network JSON file into a dict of nx.Graph objects.
    Preserves edge weights and converts timestamps like 't_1353303380' to integers.
    """
    if not os.path.exists(filename):
        raise FileNotFoundError(f"{filename} not found!")

    with open(filename, "r") as f:
        raw_data = json.load(f)

    graphs = {}
    for t_str, net in raw_data.items():
        # Convert timestamp: 't_1353303380' -> 1353303380
        timestamp = int(t_str.lstrip("t_"))

        G = nx.Graph()
        nodes = net.get("nodes", [])
        G.add_nodes_from([n["id"] for n in nodes])
        links = net.get("links", [])
        for link in links:
            src = link["source"]
            tgt = link["target"]
            w = link.get("weight", 1.0)
            G.add_edge(src, tgt, weight=w)
        
        graphs[timestamp] = G

    print(f"Loaded {len(graphs)} timesteps from {filename}")
    return graphs


In [12]:
graphs = load_real_network("data/contact_pattern_2012.json")

# Jupyter interactive version
nrugs.interactive_rug(graphs)


Loaded 11273 timesteps from data/contact_pattern_2012.json


Global stats: ((1, 1), (1, 5), (0, 4))
num_artists 2
